In [1]:
import pyodbc
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor

import joblib

c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
server = 'mia10.database.windows.net'
database = 'mia10_db'
username = 'user_reader'
password = '7R&o&o4#~756^z'
driver = '{ODBC Driver 18 for SQL Server}'

connection_string = f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}'

conn = pyodbc.connect(connection_string)

query = '''
SELECT m.slug_game, m.event_title, m.medal_type, c.country_name, e.game_year
FROM medals m
JOIN events e ON LOWER(REPLACE(m.slug_game, '-', ' ')) = LOWER(e.game_name)
JOIN country c ON m.athlete_country_id = c.country_id
WHERE e.game_season = 'Summer'
'''

df = pd.read_sql(query, conn, index_col=None)

conn.close()

C:\Users\kirit\AppData\Local\Temp\ipykernel_17468\945880379.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col=None)


In [3]:
countries_rio_2016 = df[df['slug_game'] == 'rio-2016']['country_name'].unique()
df_filtered = df[df['country_name'].isin(countries_rio_2016)]

# grouper par pays, édition et type de médaille
grouped = df_filtered.groupby(['country_name', 'slug_game', 'medal_type']).size().unstack(fill_value=0)

grouped.columns = [f'{col}_count' for col in grouped.columns]

grouped.reset_index(inplace=True)

grouped.head(1000)

,country_name,slug_game,BRONZE_count,GOLD_count,SILVER_count
0,Argentina,athens-2004,4,0,0
1,Argentina,atlanta-1996,0,0,1
2,Argentina,barcelona-1992,2,0,0
3,Argentina,beijing-2008,2,2,0
4,Argentina,berlin-1936,2,0,0
...,...,...,...,...,...
422,United States of America,rome-1960,2,0,0
423,United States of America,seoul-1988,4,9,6
424,United States of America,sydney-2000,6,9,6
425,United States of America,tokyo-1964,3,0,7


In [4]:
# Encodage des noms de pays
encoder = OneHotEncoder(sparse_output=False)
country_encoded = encoder.fit_transform(grouped[['country_name']])
country_encoded_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country_name']))

grouped = pd.concat([grouped, country_encoded_df], axis=1) # Ajouter les colonnes encodées au DataFrame

# caractéristiques supplémentaires
grouped['total_medals'] = grouped['BRONZE_count'] + grouped['GOLD_count'] + grouped['SILVER_count']
grouped['gold_silver_diff'] = grouped['GOLD_count'] - grouped['SILVER_count']
grouped['gold_bronze_diff'] = grouped['GOLD_count'] - grouped['BRONZE_count']

# préparation des données pour random forest
def create_sequences(data, time_steps=3):
    Xs, ys = [], []
    for i in range(len(data) - time_steps):
        Xs.append(data.iloc[i:(i + time_steps), 2:].values)
        ys.append(data.iloc[i + time_steps, [3, 4, 2]].values) # assuming [3, 4, 2] are the target columns
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)

# séquences
time_steps = 3
X_seq, y_seq = create_sequences(grouped, time_steps)

# Standardiser les données
scaler = StandardScaler()
X_seq_shape = X_seq.shape
X_seq = scaler.fit_transform(X_seq.reshape(-1, X_seq.shape[-1])).reshape(X_seq_shape)

X_seq = X_seq.reshape(X_seq.shape[0], -1) # Aplatir les données

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

def objective(trial):
    # hyperparamètres
    n_estimators = trial.suggest_int('n_estimators', 100, 300)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    max_depth = trial.suggest_int('max_depth', 10, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print(f'Best Hyperparameters: {study.best_params}')

best_model = RandomForestRegressor(
    **study.best_params,
    random_state=42
)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# évaluer
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

[I 2024-05-31 11:05:36,004] A new study created in memory with name: no-name-806e97b2-a91d-456c-b4f1-b5768ed93664
[I 2024-05-31 11:05:36,461] Trial 0 finished with value: 5.854260945238601 and parameters: {'n_estimators': 273, 'max_features': 'log2', 'max_depth': 16, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 0 with value: 5.854260945238601.
[I 2024-05-31 11:05:36,809] Trial 1 finished with value: 5.546066245352674 and parameters: {'n_estimators': 183, 'max_features': 'sqrt', 'max_depth': 27, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 1 with value: 5.546066245352674.
[I 2024-05-31 11:05:37,114] Trial 2 finished with value: 5.692829263097653 and parameters: {'n_estimators': 211, 'max_features': 'sqrt', 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 1 with value: 5.546066245352674.
[I 2024-05-31 11:05:37,347] Trial 3 finished with value: 5.389549625837216 and parameters: {'n_estimators': 141, 'max_features': 'log2', 'm

Best Hyperparameters: {'n_estimators': 103, 'max_features': 'log2', 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 1}
Mean Squared Error: 5.197690051874269
R^2 Score: 0.23340170851674538


In [5]:
def predict_medals_next_edition(country, model, scaler, data, encoder, time_steps=3):
    # encodage du pays
    country_encoded = encoder.transform([[country]])
    country_encoded_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country_name']))
    
    # Sélectionner les données du pays
    country_data = data[data['country_name'] == country].sort_values(by='slug_game').iloc[-time_steps:, 2:]
    country_data = pd.concat([country_data.reset_index(drop=True), country_encoded_df], axis=1)
    
    if country_data.shape[0] < time_steps:
        raise ValueError(f"Not enough data for {country} to create a prediction with {time_steps} time steps.")
    
    # Standardiser les données
    expected_features = scaler.n_features_in_
    current_features = country_data.shape[1]
    
    if current_features < expected_features:
        # ajout des 0 si nécessaire
        missing_features = expected_features - current_features
        country_data = np.hstack([country_data.values, np.zeros((country_data.shape[0], missing_features))])
    elif current_features > expected_features:
        # sup des colonnes si nécessaire
        country_data = country_data.iloc[:, :expected_features]

    country_data = scaler.transform(country_data)
    
    # aplatir les données
    country_data = country_data.reshape(1, -1)
    
    predicted_medals = model.predict(country_data)
    return pd.DataFrame(predicted_medals, columns=['GOLD_count', 'SILVER_count', 'BRONZE_count'])

In [6]:
def predict_all_countries_next_edition(model, scaler, data, encoder, time_steps=3):
    unique_countries = data['country_name'].unique()
    predictions = []

    for country in unique_countries:
        try:
            predicted_medals = predict_medals_next_edition(country, model, scaler, data, encoder, time_steps)
            predicted_medals['country_name'] = country
            predictions.append(predicted_medals)
        except ValueError as e:
            print(f"Skipping country {country}: {e}")

    predictions_df = pd.concat(predictions).reset_index(drop=True)
    return predictions_df

predictions_df = predict_all_countries_next_edition(best_model, scaler, grouped, encoder)

total_predicted_medals = predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum().sum()

# calculer le facteur de normalisation pour ajuster les médailles à la somme cible (987)
medal_target = 987
normalization_factor = medal_target / total_predicted_medals

# appliquer aux prédictions
predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']] *= normalization_factor

# arrondi entier le plus proche
predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']] = predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']].round().astype(int)

predictions_df['total_medals'] = predictions_df[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum(axis=1)

# sort par nombre total de médailles si égalité prendre or
predictions_df = predictions_df.sort_values(by=['total_medals', 'GOLD_count'], ascending=False)

predictions_df.head(50)

c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

Skipping country Lithuania: Not enough data for Lithuania to create a prediction with 3 time steps.


c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

,GOLD_count,SILVER_count,BRONZE_count,country_name,total_medals
22,49,14,14,Norway,77
35,24,22,17,United States of America,63
10,15,17,17,Germany,49
23,20,12,16,People's Republic of China,48
25,12,19,15,ROC,46
0,11,9,16,Argentina,36
27,11,12,11,Romania,34
11,5,13,13,Great Britain,31
31,7,16,6,South Africa,29
34,6,9,14,Ukraine,29


In [7]:
# sauvegarde modèle Random Forest
# joblib.dump(best_model, 'model.pkl')

## Métriques SVM

In [8]:
y_train_gold = y_train[:, 0]
y_train_silver = y_train[:, 1]
y_train_bronze = y_train[:, 2]

y_test_gold = y_test[:, 0]
y_test_silver = y_test[:, 1]
y_test_bronze = y_test[:, 2]

# modèles SVM
svr_gold = make_pipeline(StandardScaler(), SVR(C=1.0, kernel='rbf', gamma='scale'))
svr_silver = make_pipeline(StandardScaler(), SVR(C=1.0, kernel='rbf', gamma='scale'))
svr_bronze = make_pipeline(StandardScaler(), SVR(C=1.0, kernel='rbf', gamma='scale'))

# entraînement
svr_gold.fit(X_train, y_train[:, 0])
svr_silver.fit(X_train, y_train[:, 1])
svr_bronze.fit(X_train, y_train[:, 2])

# prédictions
y_pred_svr_gold = svr_gold.predict(X_test)
y_pred_svr_silver = svr_silver.predict(X_test)
y_pred_svr_bronze = svr_bronze.predict(X_test)

print(f'SVM Gold Mean Squared Error: {mean_squared_error(y_test[:, 0], y_pred_svr_gold)}')
print(f'SVM Gold R^2 Score: {r2_score(y_test[:, 0], y_pred_svr_gold)}')

print(f'SVM Silver Mean Squared Error: {mean_squared_error(y_test[:, 1], y_pred_svr_silver)}')
print(f'SVM Silver R^2 Score: {r2_score(y_test[:, 1], y_pred_svr_silver)}')

print(f'SVM Bronze Mean Squared Error: {mean_squared_error(y_test[:, 2], y_pred_svr_bronze)}')
print(f'SVM Bronze R^2 Score: {r2_score(y_test[:, 2], y_pred_svr_bronze)}')

# y_pred_svr = np.stack([y_pred_gold, y_pred_silver, y_pred_bronze], axis=1)

SVM Gold Mean Squared Error: 12.093178301278577
SVM Gold R^2 Score: 0.002816555368451046
SVM Silver Mean Squared Error: 5.834231225489424
SVM Silver R^2 Score: -0.07850582652272275
SVM Bronze Mean Squared Error: 4.167223965546632
SVM Bronze R^2 Score: 0.08890051043290237


## Métriques MLP

In [9]:
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)

y_pred_mlp = mlp_model.predict(X_test)
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print(f'MLP Mean Squared Error: {mse_mlp}')
print(f'MLP R^2 Score: {r2_mlp}')

MLP Mean Squared Error: 8.331814765930176
MLP R^2 Score: -0.3261231053106178


c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## Prédictions SVM

In [10]:
def predict_medals_next_edition_svr(country, models, scaler, data, encoder, time_steps=3):
    country_encoded = encoder.transform([[country]])
    country_encoded_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country_name']))
    
    country_data = data[data['country_name'] == country].sort_values(by='slug_game').iloc[-time_steps:, 2:]
    country_data = pd.concat([country_data.reset_index(drop=True), country_encoded_df], axis=1)
    
    if country_data.shape[0] < time_steps:
        raise ValueError(f"Not enough data for {country} to create a prediction with {time_steps} time steps.")
    
    expected_features = scaler.n_features_in_
    current_features = country_data.shape[1]
    
    if current_features < expected_features:
        missing_features = expected_features - current_features
        country_data = np.hstack([country_data.values, np.zeros((country_data.shape[0], missing_features))])
    elif current_features > expected_features:
        country_data = country_data.iloc[:, :expected_features]

    country_data = scaler.transform(country_data)
    country_data = country_data.reshape(1, -1)
    
    predicted_gold = models['gold'].predict(country_data)
    predicted_silver = models['silver'].predict(country_data)
    predicted_bronze = models['bronze'].predict(country_data)
    
    predicted_medals = np.stack([predicted_gold, predicted_silver, predicted_bronze], axis=1)
    return pd.DataFrame(predicted_medals, columns=['GOLD_count', 'SILVER_count', 'BRONZE_count'])

def predict_all_countries_next_edition_svr(models, scaler, data, encoder, time_steps=3):
    unique_countries = data['country_name'].unique()
    predictions = []

    for country in unique_countries:
        try:
            predicted_medals = predict_medals_next_edition_svr(country, models, scaler, data, encoder, time_steps)
            predicted_medals['country_name'] = country
            predictions.append(predicted_medals)
        except ValueError as e:
            print(f"Skipping country {country}: {e}")

    predictions_df = pd.concat(predictions).reset_index(drop=True)
    return predictions_df

svm_models = {'gold': svr_gold, 'silver': svr_silver, 'bronze': svr_bronze}

# Prédictions pour SVM
predictions_df_svr = predict_all_countries_next_edition_svr(svm_models, scaler, grouped, encoder)

total_predicted_medals_svr = predictions_df_svr[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum().sum()
normalization_factor_svr = medal_target / total_predicted_medals_svr
predictions_df_svr[['GOLD_count', 'SILVER_count', 'BRONZE_count']] *= normalization_factor_svr
predictions_df_svr[['GOLD_count', 'SILVER_count', 'BRONZE_count']] = predictions_df_svr[['GOLD_count', 'SILVER_count', 'BRONZE_count']].round().astype(int)
predictions_df_svr['total_medals'] = predictions_df_svr[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum(axis=1)
predictions_df_svr = predictions_df_svr.sort_values(by=['total_medals', 'GOLD_count'], ascending=False)
predictions_df_svr.head(50)


c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

Skipping country Lithuania: Not enough data for Lithuania to create a prediction with 3 time steps.


c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

,GOLD_count,SILVER_count,BRONZE_count,country_name,total_medals
25,15,22,22,ROC,59
35,24,16,16,United States of America,56
11,14,21,18,Great Britain,53
23,15,8,21,People's Republic of China,44
27,16,14,10,Romania,40
9,11,10,17,France,38
13,10,11,15,Hungary,36
26,13,6,15,Republic of Korea,34
10,10,16,8,Germany,34
32,9,9,15,Spain,33


In [11]:
# Sauvegarde des modèles SVM
joblib.dump(svr_gold, 'svr_gold_model.pkl')
joblib.dump(svr_silver, 'svr_silver_model.pkl')
joblib.dump(svr_bronze, 'svr_bronze_model.pkl')

['svr_bronze_model.pkl']

## Prédictions MLP

In [12]:
predictions_df_mlp = predict_all_countries_next_edition(mlp_model, scaler, grouped, encoder)

# prédictions MLP
total_predicted_medals_mlp = predictions_df_mlp[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum().sum()
normalization_factor_mlp = medal_target / total_predicted_medals_mlp
predictions_df_mlp[['GOLD_count', 'SILVER_count', 'BRONZE_count']] *= normalization_factor_mlp
predictions_df_mlp[['GOLD_count', 'SILVER_count', 'BRONZE_count']] = predictions_df_mlp[['GOLD_count', 'SILVER_count', 'BRONZE_count']].round().astype(int)
predictions_df_mlp['total_medals'] = predictions_df_mlp[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum(axis=1)
predictions_df_mlp = predictions_df_mlp.sort_values(by=['total_medals', 'GOLD_count'], ascending=False)
predictions_df_mlp.head(50)

c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

Skipping country Lithuania: Not enough data for Lithuania to create a prediction with 3 time steps.


,GOLD_count,SILVER_count,BRONZE_count,country_name,total_medals
35,54,13,30,United States of America,97
22,51,22,16,Norway,89
0,28,17,37,Argentina,82
27,17,28,16,Romania,61
10,1,36,20,Germany,57
25,11,28,17,ROC,56
31,10,33,0,South Africa,43
2,14,13,6,Austria,33
17,2,9,20,Japan,31
11,3,14,13,Great Britain,30


In [13]:
joblib.dump(mlp_model, 'mlp_model.pkl')

['mlp_model.pkl']

## Prédictions fusionnées

In [14]:
# prédictions Random Forest
y_pred_rf = best_model.predict(X_test)

# prédictions SVM
y_pred_svr_gold = svr_gold.predict(X_test)
y_pred_svr_silver = svr_silver.predict(X_test)
y_pred_svr_bronze = svr_bronze.predict(X_test)
y_pred_svr = np.stack([y_pred_svr_gold, y_pred_svr_silver, y_pred_svr_bronze], axis=1)

# prédictions MLP
y_pred_mlp = mlp_model.predict(X_test)

In [15]:
# moyenne simple
y_pred_ensemble = (y_pred_rf + y_pred_svr + y_pred_mlp) / 3

# moyenne pondéré
weights = {'rf': 0.5, 'svr': 0.25, 'mlp': 0.25}
y_pred_ensemble = (weights['rf'] * y_pred_rf + weights['svr'] * y_pred_svr + weights['mlp'] * y_pred_mlp)

In [16]:
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)

print(f'Ensemble Mean Squared Error: {mse_ensemble}')
print(f'Ensemble R^2 Score: {r2_ensemble}')

Ensemble Mean Squared Error: 5.374588132734125
Ensemble R^2 Score: 0.20267301021346854


In [17]:
def predict_medals_next_edition_ensemble(country, models, scaler, data, encoder, time_steps=3):
    country_encoded = encoder.transform([[country]])
    country_encoded_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(['country_name']))

    country_data = data[data['country_name'] == country].sort_values(by='slug_game').iloc[-time_steps:, 2:]
    country_data = pd.concat([country_data.reset_index(drop=True), country_encoded_df], axis=1)

    if country_data.shape[0] < time_steps:
        raise ValueError(f"Not enough data for {country} to create a prediction with {time_steps} time steps.")

    expected_features = scaler.n_features_in_
    current_features = country_data.shape[1]

    if current_features < expected_features:
        missing_features = expected_features - current_features
        country_data = np.hstack([country_data.values, np.zeros((country_data.shape[0], missing_features))])
    elif current_features > expected_features:
        country_data = country_data.iloc[:, :expected_features]

    country_data = scaler.transform(country_data)
    country_data = country_data.reshape(1, -1)

    predicted_rf = models['rf'].predict(country_data)
    predicted_svr_gold = models['svr_gold'].predict(country_data)
    predicted_svr_silver = models['svr_silver'].predict(country_data)
    predicted_svr_bronze = models['svr_bronze'].predict(country_data)
    predicted_svr = np.stack([predicted_svr_gold, predicted_svr_silver, predicted_svr_bronze], axis=1)
    predicted_mlp = models['mlp'].predict(country_data)

    predicted_ensemble = (weights['rf'] * predicted_rf + weights['svr'] * predicted_svr + weights['mlp'] * predicted_mlp)

    return pd.DataFrame(predicted_ensemble, columns=['GOLD_count', 'SILVER_count', 'BRONZE_count'])

def predict_all_countries_next_edition_ensemble(models, scaler, data, encoder, time_steps=3):
    unique_countries = data['country_name'].unique()
    predictions = []

    for country in unique_countries:
        try:
            predicted_medals = predict_medals_next_edition_ensemble(country, models, scaler, data, encoder, time_steps)
            predicted_medals['country_name'] = country
            predictions.append(predicted_medals)
        except ValueError as e:
            print(f"Skipping country {country}: {e}")

    predictions_df = pd.concat(predictions).reset_index(drop=True)
    return predictions_df

# modèles à utiliser
models_ensemble = {
    'rf': best_model,
    'svr_gold': svr_gold,
    'svr_silver': svr_silver,
    'svr_bronze': svr_bronze,
    'mlp': mlp_model
}

# prédictions
predictions_df_ensemble = predict_all_countries_next_edition_ensemble(models_ensemble, scaler, grouped, encoder)

total_predicted_medals_ensemble = predictions_df_ensemble[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum().sum()
normalization_factor_ensemble = medal_target / total_predicted_medals_ensemble
predictions_df_ensemble[['GOLD_count', 'SILVER_count', 'BRONZE_count']] *= normalization_factor_ensemble
predictions_df_ensemble[['GOLD_count', 'SILVER_count', 'BRONZE_count']] = predictions_df_ensemble[['GOLD_count', 'SILVER_count', 'BRONZE_count']].round().astype(int)
predictions_df_ensemble['total_medals'] = predictions_df_ensemble[['GOLD_count', 'SILVER_count', 'BRONZE_count']].sum(axis=1)
predictions_df_ensemble = predictions_df_ensemble.sort_values(by=['total_medals', 'GOLD_count'], ascending=False)

predictions_df_ensemble.head(50)

c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

Skipping country Lithuania: Not enough data for Lithuania to create a prediction with 3 time steps.


c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\kirit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature na

,GOLD_count,SILVER_count,BRONZE_count,country_name,total_medals
35,32,19,20,United States of America,71
22,41,15,12,Norway,68
25,12,22,17,ROC,51
10,10,22,16,Germany,48
0,14,11,21,Argentina,46
27,13,17,12,Romania,42
23,16,8,17,People's Republic of China,41
11,6,14,14,Great Britain,34
31,8,20,3,South Africa,31
9,4,6,17,France,27
